In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.17.2
pandas 0.25.3
sklearn 0.21.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
t = tf.constant([[1.,2.,3.], [4.,5.,6.]])
# index 操作
print(t)
print(t[:,1:])
print(t[..., 1])
print(t[:, 1])

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


In [3]:
# op操作
print(t * 10)
print(tf.sqrt(t))
print(t @ tf.transpose(t)) # 矩阵乘法

tf.Tensor(
[[10. 20. 30.]
 [40. 50. 60.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1.        1.4142135 1.7320508]
 [2.        2.236068  2.4494898]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [4]:
# numpy conversion
print(t.numpy())
print(np.square(t))
np_t = np.array([[1.,2.,3.], [4.,5.,6.]])
print(tf.constant(np_t))

[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


In [5]:
t

<tf.Tensor: id=0, shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [9]:
# scalars
t = tf.constant(123456)
print(t.numpy())
print(t.shape)
print(tf.constant([123456]))
print(tf.constant([123456, ]))

123456
()
tf.Tensor([123456], shape=(1,), dtype=int32)
tf.Tensor([123456], shape=(1,), dtype=int32)


In [10]:
# strings
s = tf.constant('coffee')
print(s)
print(tf.strings.length(s))
print(tf.strings.length(s, unit='UTF8_CHAR'))
print(tf.strings.unicode_decode(s, 'UTF8'))

tf.Tensor(b'coffee', shape=(), dtype=string)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor([ 99 111 102 102 101 101], shape=(6,), dtype=int32)


In [11]:
# string array
s = tf.constant(['cafe', 'coffee', '咖啡'])
print(tf.strings.length(s))
print(tf.strings.length(s, unit='UTF8_CHAR'))
r = tf.strings.unicode_decode(s, "UTF8")
print(r)

tf.Tensor([4 6 6], shape=(3,), dtype=int32)
tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


In [12]:
# ragged tensor
r = tf.ragged.constant([[11, 12], [21, 22, 23], [], [41]])
# index  操作
print(r)
print(r[1])
print(r[:1])

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[11, 12]]>


In [13]:
# ops on ragged tensor
r2 = tf.ragged.constant([[51,52],[],[71]])  
print(tf.concat([r, r2], axis=0))  # 行拼接

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41], [51, 52], [], [71]]>


In [14]:
r3 = tf.ragged.constant([[13, 14], [24], [0], [42,43]])
print(tf.concat([r, r3], axis=1)) # 列拼接，前提条件是行数一样

<tf.RaggedTensor [[11, 12, 13, 14], [21, 22, 23, 24], [0], [41, 42, 43]]>


In [15]:
print(r.to_tensor()) # 将RaggedTensor转为Tensor

tf.Tensor(
[[11 12  0]
 [21 22 23]
 [ 0  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)


In [16]:
# sparse tensor
s = tf.SparseTensor(indices=[[0,1], [1,0],[2,3]],
                   values=[1., 2., 3.],
                   dense_shape=[3,4])
print(s)
print(tf.sparse.to_dense(s))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [17]:
# ops on sparse tensor
s2 = s * 2.0
print(s2)

try:
    s3 = s - 1 # 稀疏张量不能进行加减法
except TypeError as ex:
    print(ex)

s4 = tf.constant([[10., 20.],
                 [30., 40.],
                 [50., 60.],
                 [70., 80.]])
print(tf.sparse.sparse_dense_matmul(s, s4))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for -: 'SparseTensor' and 'int'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]], shape=(3, 2), dtype=float32)


In [20]:
# sparse tensor
s5 = tf.SparseTensor(indices=[[1,0], [1,0],[2,3]],
                   values=[1., 2., 3.],
                   dense_shape=[3,4])
print(s5)
print(tf.sparse.to_dense(s5)) # 稀疏张量转为密集张量时的索引必须是排过序的

SparseTensor(indices=tf.Tensor(
[[1 0]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


InvalidArgumentError: indices[1] = [1,0] is repeated [Op:SparseToDense]

In [21]:
# sparse tensor
s5 = tf.SparseTensor(indices=[[1,1], [1,0],[2,3]],
                   values=[1., 2., 3.],
                   dense_shape=[3,4])
print(s5)
s6 = tf.sparse.reorder(s5) # 排好序才能转化为密集张量
print(tf.sparse.to_dense(s6))

SparseTensor(indices=tf.Tensor(
[[1 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 0. 0. 0.]
 [2. 1. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [22]:
# variable
v = tf.Variable([[1.,2.,3.], [4.,5.,6.]])
print(v)
print(v.value())
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


In [23]:
# assign value 
v.assign(2*v)
print(v.numpy())
v[0, 1].assign(100)
print(v.numpy())
v[0].assign([7.,8.,9.])
print(v.numpy())

[[ 2.  4.  6.]
 [ 8. 10. 12.]]
[[  2. 100.   6.]
 [  8.  10.  12.]]
[[ 7.  8.  9.]
 [ 8. 10. 12.]]


In [24]:
try:
    v[1] = [7.,8.,9.]  # Variable只能通过assign赋值
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment
